# Extending FBA with modifications

It is often desirable to add a slight modification to the problem before
performing the analysis, to see e.g. differences of the model behavior caused
by the change introduced.

First, let us load everything that will be required:

In [1]:
!isfile("e_coli_core.xml") &&
    download("http://bigg.ucsd.edu/static/models/e_coli_core.xml", "e_coli_core.xml")

using COBREXA, GLPK, Tulip, JuMP

model = load_model("e_coli_core.xml")

Metabolic model of type SBMLModel
sparse([41, 23, 51, 67, 61, 65, 1, 7, 19, 28  …  72, 3, 8, 33, 57, 66, 31, 45, 46, 57], [1, 2, 2, 2, 3, 3, 4, 4, 4, 4  …  93, 94, 94, 94, 94, 94, 95, 95, 95, 95], [-1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0  …  1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 1.0], 72, 95)
Number of reactions: 95
Number of metabolites: 72


`COBREXA.jl` supports [many modifications](../concepts/2_modifications.md),
which include changing objective sense, optimizer attributes, flux
constraints, optimization objective, reaction and gene knockouts, and others.
These modifications are applied to the optimization built within the supplied
optimizer (in this case GLPK) in order as they are specified. User needs to
manually ensure that the modification ordering is sensible.

The following example applies multiple different modifications to the *E.
coli* core model:

In [2]:
fluxes = flux_balance_analysis_dict(
    model,
    GLPK.Optimizer;
    modifications = [ # modifications are applied in order
        change_objective("R_BIOMASS_Ecoli_core_w_GAM"), # maximize production
        change_constraint("R_EX_glc__D_e"; lb = -12, ub = -12), # fix an exchange rate
        knockout(["b0978", "b0734"]), # knock out two genes
        change_optimizer(Tulip.Optimizer), # ignore the above optimizer and switch to Tulip
        change_optimizer_attribute("IPM_IterationsLimit", 1000), # customize Tulip
        change_sense(JuMP.MAX_SENSE), # explicitly tell Tulip to maximize the objective
    ],
)

Dict{String, Float64} with 95 entries:
  "R_EX_fum_e"    => 0.0
  "R_ACONTb"      => 7.03277
  "R_TPI"         => 8.90908
  "R_SUCOAS"      => -5.8921
  "R_GLNS"        => 0.270339
  "R_EX_pi_e"     => -3.88931
  "R_PPC"         => 3.02966
  "R_O2t"         => 25.7859
  "R_G6PDH2r"     => 6.11782
  "R_TALA"        => 1.85013
  "R_PPCK"        => 5.26409e-10
  "R_EX_lac__D_e" => 4.37341e-12
  "R_PGL"         => 6.11782
  "R_H2Ot"        => -34.7096
  "R_GLNabc"      => 0.0
  "R_EX_co2_e"    => 27.0082
  "R_EX_gln__L_e" => 0.0
  "R_EX_nh4_e"    => -5.76498
  "R_MALt2_2"     => 0.0
  ⋮               => ⋮

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*